In [1]:
import pandas as pd
from datetime import timedelta

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 700)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

# Import & process SOD-labeled data
This data will be used to label the following unlabeled data:
1. Pre-labeled as head
2. Predicted as head (will require evaluation to ensure data only consists of heads)

In [2]:
df_SOD_labeled = pd.read_csv('../data/original/stages.csv.20221114_correct.processed', header=None, delimiter='/',
                    usecols=[8])
display(df_SOD_labeled.head())

df_SOD_labeled[['file','label']] = df_SOD_labeled[8].str.split(',', expand=True)
# remove duplicates
df_SOD_labeled.drop_duplicates(subset='file', keep="first", inplace=True)
df_SOD_labeled['file'] = df_SOD_labeled['file'].str.replace(' ','')
df_SOD_labeled[['id_date', 'count','ext']] = df_SOD_labeled['file'].str.split('.', expand=True)
df_SOD_labeled['id'] = df_SOD_labeled['id_date'].str.slice(stop=3)
df_SOD_labeled['yrs_in_fac'] = df_SOD_labeled['id_date'].str.slice(start=4, stop=5)
df_SOD_labeled['month_day'] = df_SOD_labeled['id_date'].str.slice(start=4)
df_SOD_labeled.drop([8,'ext'], axis=1, inplace=True)
display(df_SOD_labeled.head())
print(df_SOD_labeled.shape)
print(df_SOD_labeled.info())
display(df_SOD_labeled.describe())

,8
0,"00000122.08.JPG,0"
1,"00000129.14.JPG,0"
2,"00000213.21.JPG,1"
3,"00000219.07.JPG,1"
4,"00000222.08.JPG,2"


,file,label,id_date,count,id,yrs_in_fac,month_day
0,00000122.08.JPG,0,00000122,08,000,0,0122
1,00000129.14.JPG,0,00000129,14,000,0,0129
2,00000213.21.JPG,1,00000213,21,000,0,0213
3,00000219.07.JPG,1,00000219,07,000,0,0219
4,00000222.08.JPG,2,00000222,08,000,0,0222


(4706, 7)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4706 entries, 0 to 4730
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   file        4706 non-null   object
 1   label       4706 non-null   object
 2   id_date     4706 non-null   object
 3   count       4706 non-null   object
 4   id          4706 non-null   object
 5   yrs_in_fac  4706 non-null   object
 6   month_day   4706 non-null   object
dtypes: object(7)
memory usage: 294.1+ KB
None


,file,label,id_date,count,id,yrs_in_fac,month_day
count,4706,4706,4706,4706,4706,4706,4706
unique,4706,4,3951,90,653,2,366
top,00000122.08.JPG,0,ff600511,07,fc0,0,0409
freq,1,1463,14,285,305,3714,39


In [3]:
# create 'year' column from 'yrs_in_fac' column and then a 'date' column
def assign_year(row):
    if row['yrs_in_fac'] == '0':
        val = '12'
    elif row['yrs_in_fac']  == '1':
        val = '13'
    return val

df_SOD_labeled['year'] = df_SOD_labeled.apply(assign_year, axis=1)  # create year column 
df_SOD_labeled['date'] = df_SOD_labeled['year']+df_SOD_labeled['month_day']  # create date column

# convert 'date' to dtype datetime
df_SOD_labeled['date'] = pd.to_datetime(df_SOD_labeled['date'], format='%y%m%d', errors='coerce')
display(df_SOD_labeled[df_SOD_labeled['date'].isnull()])
display(df_SOD_labeled.head())
print(df_SOD_labeled.info())

,file,label,id_date,count,id,yrs_in_fac,month_day,year,date


,file,label,id_date,count,id,yrs_in_fac,month_day,year,date
0,00000122.08.JPG,0,00000122,08,000,0,0122,12,2012-01-22
1,00000129.14.JPG,0,00000129,14,000,0,0129,12,2012-01-29
2,00000213.21.JPG,1,00000213,21,000,0,0213,12,2012-02-13
3,00000219.07.JPG,1,00000219,07,000,0,0219,12,2012-02-19
4,00000222.08.JPG,2,00000222,08,000,0,0222,12,2012-02-22


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4706 entries, 0 to 4730
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   file        4706 non-null   object        
 1   label       4706 non-null   object        
 2   id_date     4706 non-null   object        
 3   count       4706 non-null   object        
 4   id          4706 non-null   object        
 5   yrs_in_fac  4706 non-null   object        
 6   month_day   4706 non-null   object        
 7   year        4706 non-null   object        
 8   date        4706 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(8)
memory usage: 367.7+ KB
None


In [4]:
df_SOD_labeled2 = df_SOD_labeled[['file','label', 'id','date']].copy()
display(df_SOD_labeled2.head())
print(df_SOD_labeled2.shape)

,file,label,id,date
0,00000122.08.JPG,0,000,2012-01-22
1,00000129.14.JPG,0,000,2012-01-29
2,00000213.21.JPG,1,000,2012-02-13
3,00000219.07.JPG,1,000,2012-02-19
4,00000222.08.JPG,2,000,2012-02-22


(4706, 4)


# Import & process pre-labeled head data
Need SOD label

In [5]:
df_head_labeled = pd.read_csv('/da1_data/icputrd/decaying_human_body_part_classifier/from_anau/ex1_preds_labeled_head', 
                                header=None, delimiter=':', 
                                   usecols=[0,2])
df_head_labeled.columns =['path', 'confidence']

df_head_labeled['path'] = df_head_labeled['path'].str.replace(' ','')
df_head_labeled['file'] = df_head_labeled['path'].str.split('/').str[-1]
df_head_labeled[['id_date', 'count','ext']] = df_head_labeled['file'].str.split('.', expand=True)
df_head_labeled['id'] = df_head_labeled['id_date'].str.slice(stop=3)
df_head_labeled['yrs_in_fac'] = df_head_labeled['id_date'].str.slice(start=4, stop=5)
df_head_labeled['month_day'] = df_head_labeled['id_date'].str.slice(start=4)
df_head_labeled.drop(['path','ext'], axis=1, inplace=True)
display(df_head_labeled.head())
print(df_head_labeled.shape)
print(df_head_labeled.info())
display(df_head_labeled.describe())

,confidence,file,id_date,count,id,yrs_in_fac,month_day
0,99.99,1bc10209.36.JPG,1bc10209,36,1bc,0,0209
1,100.00,1bc10212.30.JPG,1bc10212,30,1bc,0,0212
2,100.00,1bc10213.30.JPG,1bc10213,30,1bc,0,0213
3,99.99,1bc10214.32.JPG,1bc10214,32,1bc,0,0214
4,100.00,1bc10216.31.JPG,1bc10216,31,1bc,0,0216


(12629, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12629 entries, 0 to 12628
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   confidence  12629 non-null  float64
 1   file        12629 non-null  object 
 2   id_date     12629 non-null  object 
 3   count       12629 non-null  object 
 4   id          12629 non-null  object 
 5   yrs_in_fac  12629 non-null  object 
 6   month_day   12629 non-null  object 
dtypes: float64(1), object(6)
memory usage: 690.8+ KB
None


,confidence
count,12629.000000
mean,98.857434
std,5.784056
min,29.450000
25%,99.960000
50%,100.000000
75%,100.000000
max,100.000000


In [6]:
df_head_labeled['year'] = df_head_labeled.apply(assign_year, axis=1)  # create year column 
df_head_labeled['date'] = df_head_labeled['year']+df_head_labeled['month_day']  # create date column

# convert 'date' to dtype datetime
df_head_labeled['date'] = pd.to_datetime(df_head_labeled['date'], format='%y%m%d', errors='coerce')
display(df_head_labeled[df_head_labeled['date'].isnull()])
display(df_head_labeled.head())
print(df_head_labeled.info())

,confidence,file,id_date,count,id,yrs_in_fac,month_day,year,date


,confidence,file,id_date,count,id,yrs_in_fac,month_day,year,date
0,99.99,1bc10209.36.JPG,1bc10209,36,1bc,0,0209,12,2012-02-09
1,100.00,1bc10212.30.JPG,1bc10212,30,1bc,0,0212,12,2012-02-12
2,100.00,1bc10213.30.JPG,1bc10213,30,1bc,0,0213,12,2012-02-13
3,99.99,1bc10214.32.JPG,1bc10214,32,1bc,0,0214,12,2012-02-14
4,100.00,1bc10216.31.JPG,1bc10216,31,1bc,0,0216,12,2012-02-16


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12629 entries, 0 to 12628
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   confidence  12629 non-null  float64       
 1   file        12629 non-null  object        
 2   id_date     12629 non-null  object        
 3   count       12629 non-null  object        
 4   id          12629 non-null  object        
 5   yrs_in_fac  12629 non-null  object        
 6   month_day   12629 non-null  object        
 7   year        12629 non-null  object        
 8   date        12629 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 888.1+ KB
None


In [7]:
df_head_labeled2 = df_head_labeled[['file', 'id', 'date']].copy()
display(df_head_labeled2.head())
print(df_head_labeled2.shape)

,file,id,date
0,1bc10209.36.JPG,1bc,2012-02-09
1,1bc10212.30.JPG,1bc,2012-02-12
2,1bc10213.30.JPG,1bc,2012-02-13
3,1bc10214.32.JPG,1bc,2012-02-14
4,1bc10216.31.JPG,1bc,2012-02-16


(12629, 3)


In [8]:
def label_propagation(minus_days, plus_plays):
    # create an Empty DataFrame object
    df_new = pd.DataFrame()
    # for each img 
    for row in df_SOD_labeled2.itertuples():
        # create date range
        date_minus = row[4] - timedelta(days=minus_days)
        date_plus = row[4] + timedelta(days=plus_plays)
        #print(date_minus, row[4], date_plus)
        # get all img between date range for current donor and append to new df
        #print(row[3])
        df = df_head_labeled2[df_head_labeled2['id'] == row[3]]
        if df.shape[0] == 0:
            continue
        #print(row)
        #print(date_minus, row[4], date_plus)
        #display(df)
        df_between = df.loc[df['date'].between(date_minus, date_plus)]
        if df_between.shape[0] != 0:
            #print(row)
            df_between.loc[:,'label'] = row[2] # propagate the label
            #display(df_between.head())
            # remove current img to avoid duplicates
            #print(df_between.shape)
            df_between = df_between[df_between.file != row[1]]
            #print(df_between.shape)
            df_new = pd.concat([df_new, df_between])
    
    return df_new

In [9]:
df_new = label_propagation(2,2)
print(df_new.shape)
display(df_new.head())

(9892, 4)


,file,id,date,label
1048,02d00704.36.JPG,02d,2012-07-04,0
1049,02d00704.48.JPG,02d,2012-07-04,0
1050,02d00705.41.JPG,02d,2012-07-05,0
1051,02d00705.48.JPG,02d,2012-07-05,0
1052,02d00705.49.JPG,02d,2012-07-05,0


In [10]:
# merge the propagated data with the original data
df_final = pd.concat([df_SOD_labeled[['id', 'file', 'label']], df_new[['id', 'file', 'label']]])
print(df_final.shape)
display(df_final.head())

(14598, 3)


,id,file,label
0,000,00000122.08.JPG,0
1,000,00000129.14.JPG,0
2,000,00000213.21.JPG,1
3,000,00000219.07.JPG,1
4,000,00000222.08.JPG,2


In [12]:
# drop duplicate rows 
df_final.drop_duplicates(inplace=True)
print(df_final.shape)

(11790, 3)


In [13]:
# create image path column before saving as csv
df_final['path'] = '/da1_data/icputrd/arf/mean.js/public/anau_img3/' + df_final['id'] + '/' + df_final['file']
print(df_final.shape)
display(df_final.head())

(11790, 4)


,id,file,label,path
0,000,00000122.08.JPG,0,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000122.08.JPG
1,000,00000129.14.JPG,0,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000129.14.JPG
2,000,00000213.21.JPG,1,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000213.21.JPG
3,000,00000219.07.JPG,1,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000219.07.JPG
4,000,00000222.08.JPG,2,/da1_data/icputrd/arf/mean.js/public/anau_img3/000/00000222.08.JPG


In [14]:
# save image path and label as csv ready for 03_train_val_test_split.py
df_final[['path', 'label']].to_csv('../data/propagated_2_2/stages.csv.20221114_correct.processed.propagated_2_2', header=None,
                                 index=False)